Acknowledgement: 

Tensorflow 2 official tutorial https://www.tensorflow.org/tutorials/images/transfer_learning

CS231n course website https://cs231n.github.io/

# Prepare environment

Colab is a free tool for experimentation for your project. 
Colab will disconnect after 12 hours or ~30 min of idling (and you will lose your unsaved data). So let's mount to your Google drive first.

In [158]:
import os
import tensorflow as tf
import numpy as np
#import tensorflow_hub as hub
import requests 
import gzip, shutil
import os 
import data_utils as du 
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from six.moves import cPickle as pickle
import numpy as np
import os
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt 
import platform
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 15.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Data_folder = 'CIFAR10_Data'
Data_fn = 'cifar-10-python'
cifar_url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
#Chang to true if need download data 
Down_load_data = False 
if Down_load_data:
    r = requests.get(cifar_url,allow_redirects=True)
    open(os.path.join(Data_folder,Data_fn+'.tar.gz'),'wb').write(r.content)
    #Unzip the data
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from six.moves import cPickle as pickle
import numpy as np
import os
from matplotlib.pyplot import imread
import platform

def load_pickle(f):
  version = platform.python_version_tuple()
  if version[0] == '2':
      return  pickle.load(f)
  elif version[0] == '3':
      return  pickle.load(f, encoding='latin1')
  raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(fn):
    with open(fn, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000,subtract_mean=True):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for training  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'CIFAR10_Data/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    ## Data post_process 
    
    if subtract_mean:
        mean_image = np.mean(X_train, axis=0)
        X_train -= mean_image
        X_val -= mean_image
        X_test -= mean_image
    

    return X_train, y_train, X_val, y_val, X_test, y_test
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int64
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


# Colab users

If you are using Colab, you can manually switch to a GPU device in tf.device(device) scope.

In [159]:
# Set up some global variables
USE_GPU = True

if USE_GPU:
  device = '/device:GPU:0'
else:
  device = '/cpu:0'

print('Using device: ', device)

Using device:  /device:GPU:0


# Data

Load CIFAR-10 dataset for training

In [160]:
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int64
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [161]:
class Dataset(object):
  def __init__(self, X, y, batch_size, shuffle=False):
    """
    Construct a Dataset object to iterate over data X and labels y
    
    Inputs:
    - X: Numpy array of data, of any shape
    - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
    - batch_size: Integer giving number of elements per minibatch
    - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
    """
    assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
    self.X, self.y = X, y
    self.batch_size, self.shuffle = batch_size, shuffle

  def __iter__(self):
    N, B = self.X.shape[0], self.batch_size
    idxs = np.arange(N)
    if self.shuffle:
      np.random.shuffle(idxs)
    return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [162]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


# Model

There are several ways to define the model. We use the low-level APIs in the example model below for flexibility. Check https://www.tensorflow.org/tutorials/images/classification for Sequential/Functional APIs.

In [163]:
class ThreeLayerConvNet(tf.keras.Model):
  def __init__(self, channel_1, channel_2, num_classes):
    super(ThreeLayerConvNet, self).__init__()

    initializer = tf.initializers.VarianceScaling(scale=0.002)
    self.conv1 = tf.keras.layers.Conv2D(channel_1, [5,5], [1,1], padding='valid',
                                     kernel_initializer=initializer,
                                     activation=tf.nn.relu)
    self.conv2 = tf.keras.layers.Conv2D(channel_2, [3,3], [1,1], padding='valid',
                                     kernel_initializer=initializer,
                                     activation=tf.nn.relu)
    self.fc = tf.keras.layers.Dense(num_classes, kernel_initializer=initializer)
    self.flatten = tf.keras.layers.Flatten()   
    self.softmax = tf.keras.layers.Softmax()     
    
  def call(self, x, training=False):
    scores = None

    padding = tf.constant([[0,0],[2,2],[2,2],[0,0]])
    x = tf.pad(x, padding, 'CONSTANT')
    x = self.conv1(x)
    padding = tf.constant([[0,0],[1,1],[1,1],[0,0]])
    x = tf.pad(x, padding, 'CONSTANT')
    x = self.conv2(x)
    x = self.flatten(x)
    x = self.fc(x)
    scores = self.softmax(x)
       
    return scores

In [164]:
def model_init_fn():
  ###########################################################################
  # TODO: Implement your model                     #
  ###########################################################################
  
  channel_1, channel_2, num_classes = 12, 8, 10
  model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

  ###########################################################################
  #             END OF YOUR CODE              #
  ###########################################################################
  return model

# Optimizer

In [165]:
def optimizer_init_fn():
  ###########################################################################
  # TODO: Implement your optimizer                   #
  ###########################################################################
  
  opt = tf.optimizers.SGD(learning_rate=0.001)

  ###########################################################################
  #             END OF YOUR CODE              #
  ###########################################################################
  return opt

# Loss

In [166]:
def loss_init_fn():
  ############################################################################
  # TODO: Implement your loss                      #
  ############################################################################
  
  loss = keras.losses.SparseCategoricalCrossentropy()

  ###########################################################################
  #             END OF YOUR CODE              #
  ###########################################################################
  return loss

# Train

In [167]:
def train(model, optimizer, loss_fn, num_epochs=1, save_every_num_epoch=2, save_dir='tmp/', save_model_name='trained', is_training=False):
  """
  Simple training loop for use with models defined using tf.keras. It trains
  a model for one epoch on the CIFAR-10 training set and periodically checks
  accuracy on the CIFAR-10 validation set.
  
  Inputs:
  - num_epochs: The number of epochs to train for
  - save_every_num_epoch: The number of epochs to export the trained model
  - save_dir: Directory to export the trained model
  - save_model_name: Sub-directory to export the trained model (also the name to reload the model)
  
  Returns: Nothing, but prints progress during training
  """    
  with tf.device(device):
   
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

    val_loss = tf.keras.metrics.Mean(name='val_loss')
    val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
    
    t = 0
    for epoch in range(num_epochs):      
      # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
      train_loss.reset_states()
      train_accuracy.reset_states()
      
      for x_np, y_np in train_dset:
        with tf.GradientTape() as tape:
          ###########################################################################
          # TODO: 1. Build forward pass with model             #    
          #     2. Compute the loss with loss_fn             #
          ###########################################################################
          
          scores = model(x_np)
          #print(scores)
          loss = loss_fn(y_np,scores)

          ###########################################################################
          #             END OF YOUR CODE              #
          ###########################################################################

          gradients = tape.gradient(loss, model.trainable_variables)
          optimizer.apply_gradients(zip(gradients, model.trainable_variables))
          
          # Update the metrics
          train_loss.update_state(loss)
          train_accuracy.update_state(scores,y_np)
          
          if t % 100 == 0:
            val_loss.reset_states()
            val_accuracy.reset_states()
            for test_x, test_y in val_dset:
              ###########################################################################
              # TODO: 1. Run validation with model                #    
              #     2. Compute the loss with loss_fn             #
              ###########################################################################
              
              prediction = model(test_x)
              t_loss = loss_fn(test_y,prediction)

              ###########################################################################
              #             END OF YOUR CODE              #
              ###########################################################################

              val_loss.update_state(t_loss)
              val_accuracy.update_state(test_y, prediction)
            
            template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
            print (template.format(t, epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        val_loss.result(),
                        val_accuracy.result()*100))
          t += 1
      if epoch % save_every_num_epoch or epoch == num_epochs - 1:
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        save_path = os.path.join(save_dir, save_model_name+'-'+str(epoch+1))
        tf.saved_model.save(model, save_path)

In [168]:
model = model_init_fn()
optimizer = optimizer_init_fn()
loss = loss_init_fn()
train(model, optimizer, loss)

Iteration 0, Epoch 1, Loss: 2.3024110794067383, Accuracy: 0.0, Val Loss: 2.3026750087738037, Val Accuracy: 11.59999942779541
Iteration 100, Epoch 1, Loss: 2.2164223194122314, Accuracy: 0.0, Val Loss: 2.1063730716705322, Val Accuracy: 26.80000114440918
Iteration 200, Epoch 1, Loss: 2.1237564086914062, Accuracy: 0.0, Val Loss: 1.9352222681045532, Val Accuracy: 32.20000076293945
Iteration 300, Epoch 1, Loss: 2.056257486343384, Accuracy: 0.0, Val Loss: 1.8988291025161743, Val Accuracy: 33.70000076293945
Iteration 400, Epoch 1, Loss: 1.9927791357040405, Accuracy: 0.0, Val Loss: 1.8013007640838623, Val Accuracy: 36.599998474121094
Iteration 500, Epoch 1, Loss: 1.9510821104049683, Accuracy: 0.0, Val Loss: 1.7409687042236328, Val Accuracy: 38.400001525878906
Iteration 600, Epoch 1, Loss: 1.9156017303466797, Accuracy: 0.0, Val Loss: 1.6906319856643677, Val Accuracy: 40.900001525878906
Iteration 700, Epoch 1, Loss: 1.8844273090362549, Accuracy: 0.0, Val Loss: 1.6449127197265625, Val Accuracy: 41

# Finetune

https://www.tensorflow.org/hub/tf2_saved_model?hl=en

In [169]:
model = hub.KerasLayer("tmp/trained-3", trainable = True)
# redefine the optimizer (we usually use smaller learning rate for finetuning)
train(model, optimizer, loss, num_epochs=1)

NameError: name 'hub' is not defined

# Quantize a pre-trained model

In [ ]:
model = hub.KerasLayer("tmp/trained-3", trainable=True)

In [ ]:
def my_quantize_strategy(var):
  ############################################################################
  # TODO: Implement your quantization strategy             #
  ############################################################################
  
  var = -var

  ###########################################################################
  #             END OF YOUR CODE              #
  ###########################################################################
  return var

In [ ]:
for var in model.trainable_variables:
  var.assign(my_quantize_strategy(var))